In [1]:
import os
import ee
import geemap
import time
import random
from datetime import datetime, timedelta , date
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
#Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 
Map = geemap.Map()

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [46]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [23]:
# give the bands

band_name = widgets.Dropdown(
options=['LST_Day','LST_Night','LST_Mean'],
description='Band')

band_name

Dropdown(description='Band', options=('LST_Day', 'LST_Night', 'LST_Mean'), value='LST_Day')

In [24]:
# gine the date frequency

frequency =widgets.Dropdown(
    options=['Year','Month'],
    value='Year',
    description='Frequency')

frequency

Dropdown(description='Frequency', options=('Year', 'Month'), value='Year')

In [25]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\USER\Desktop\satellite python code\satellite python code\Geotif', filename='', titl…

In [26]:
# give the scale number
scale = widgets.IntSlider(
    value=1000,
    min=500,
    max=10000,
    step=100,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

scale

IntSlider(value=1000, continuous_update=False, description='Scale:', max=10000, min=500, step=100)

In [27]:
roi = geemap.shp_to_ee(''.join(my_button.files)).geometry()

In [28]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [29]:
# This is the function after recalculating the bands

def lst_filter(image):
    
    qaday = image.select(['QC_Day']); 
    qanight = image.select(['QC_Night']); 
    dayshift = qaday.rightShift(6);
    nightshift = qanight.rightShift(6);
    daymask = dayshift.lte(2);
    nightmask = nightshift.lte(2);
    #dayshift1 = qaday.leftShift(6);
    #dayshift2 = qaday.rightShift(6);
    #daymask = dayshift2.eq(0);
    #nightshift1 = qanight.leftShift(6);
    #nightshift2 = qanight.rightShift(6);
    #nightmask = nightshift2.eq(0);  
    outimage = ee.Image(image.select(['LST_Day_1km', 'LST_Night_1km']));
    outmask = ee.Image([daymask, nightmask]);
    return outimage.updateMask(outmask); 

def lst_day(image):
    
    LST_Day = image.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename("LST_Day")
    image = image.addBands(LST_Day)

    return(image)

def lst_night(image):
    
    LST_Night = image.select('LST_Night_1km').multiply(0.02).subtract(273.15).rename("LST_Night")
    image = image.addBands(LST_Night)

    return(image)

def lst_mean(image):
    LST_Mean = image.expression(
    '(day + night) / 2', {
    'day': image.select('LST_Day'),
    'night': image.select('LST_Night')}).rename('LST_Mean');

    return image.addBands(LST_Mean);

In [30]:
def geotif(time_list):
   
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        dataset = ee.ImageCollection("MODIS/061/MOD11A2") \
                      .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                      .filterBounds(roi) \
                      .map(lst_filter) \
                      .map(lst_day) \
                      .map(lst_night) \
                      .map(lst_mean) \
                      .select(band_name.value) \
                      .mean()
    
        image = dataset.clip(roi).unmask()
        filename = os.path.join(os.path.expanduser(folder.selected) + '_{}.tif'.format(time_list[i]))
        geemap.ee_export_image(
            image, filename=filename, scale=scale.value, region=roi, file_per_band=False
                                )

In [35]:
if 'Year' in frequency.value.strip():
    time_list = yearlist(star.value, end.value)
    geotif(time_list)
else:
    pass
if "Month" in frequency.value.strip():
    time_list =monthlist(star.value, end.value)
    geotif(time_list)
else:
    pass

Generating URL ...
Please wait ...
Data downloaded to D:\5566_['2020-01-01', '2020-01-31'].tif
Generating URL ...
Please wait ...
Data downloaded to D:\5566_['2020-02-01', '2020-02-11'].tif


In [40]:
w = widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)
w

SelectMultiple(description='Fruits', index=(1,), options=('Apples', 'Oranges', 'Pears'), value=('Oranges',))

In [43]:
w.value

('Apples', 'Oranges', 'Pears')